In [ ]:
import pymysql
from mysql.connector import Error
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

In [ ]:
#Connect to MySQL database
def connect_to_db(host, user, password):
    try:
        conn = pymysql.connect(   
            host= host,
            user= user,
            password=password
        )
    except Error as e:
        print(e)
    else:
        print('Connected!')
    return conn

In [ ]:
#Database credentials
host='xxxxxxxxx.rds.amazonaws.com'
user='xxxxxx'
password='xxxxxx'
conn = None

#Connecting to the db
conn = connect_to_db(host, user, password)
#Allowing us to execute SQL commands
curr = conn.cursor()

In [ ]:
use_database = ("""USE xxxxxx;""")
curr.execute(use_database)

In [ ]:
curr.execute("SELECT average_height, date, BTC_fees, USD_fees FROM block_fees")
result = curr.fetchall()

In [ ]:
average_height = []
date = []
BTC_fees = []
USD_fees = []

for i in result:
    average_height.append(i[0])
    date.append(i[1])
    BTC_fees.append(i[2])
    USD_fees.append(i[3])

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=date, y=BTC_fees,name="Fees BTC", hovertemplate ='%{y} BTC', line_color='#a69cac',),            
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=date, y=USD_fees,customdata=average_height, name="Fees USD", hovertemplate ='%{y}' + '<br>Avg block: %{customdata}', line_color='#6610f2',),
    secondary_y=True,)

fig.update_layout(
    paper_bgcolor="#11131f",
    plot_bgcolor="#11131f",
    font_color="white",
    margin_r=5,
    title=dict(
        text="Bitcoin Block Fees - 1M",
        font_family="Microsoft YaHei",
        font_size=20,
        x=0.50,
        y=0.91,
        xanchor='center',
        yanchor='top'
    ),
    #combining the hoverlabel for the 2 yaxes
    hovermode="x unified",
    hoverlabel=dict(
        bgcolor="#11131f",
        font_size=16,
        font_family="Microsoft YaHei",
        bordercolor="black",
        font_color='#6c757d',
    ),
    legend=dict(
        font_color="#6c757d",
         x=0.39,
         y=0.95,
        orientation="h",
    )
)

#setting up the dates on the xaxis
fig.update_xaxes(
    title_text="",
    showgrid=False,
    #show all days in the xaxis; 1-31
    tickmode = 'linear',
    #make a step between the values in the xaxis
    #that = to 2 days in milliseconds
    dtick=172800000.0,
    #showing only the day
    tickformat='%d',
    color='#6c757d',
    showline=True,
    linecolor='#e0fbfc',
    gridcolor='#6c757d', 
    #make the rangeslider visible
    rangeselector=dict(
           buttons=list([
               dict(count=1,
                    step="day",
                    stepmode="backward",
                    visible=False
                   ),
           ])
       ),
    rangeslider=dict(
          visible=True
        ), 
)

#customizing the first yaxis
fig.update_yaxes(
    title_text="BTC",
    secondary_y=False,
    #setting the step between the values to be 0.1
    dtick=0.1,
    color='#6c757d',
    gridcolor='#6E7079'
    

)
#customizing the second yaxis
fig.update_yaxes(
    title_text="USD",
    #adding a $ sign in front of the values of the secondary yaxis
    tickformat="$",
    secondary_y=True,
    #hiding the grid lines of the secondary yaxis
    showgrid=False,
    color='#6c757d',
    
)

fig.update_traces(
    #set the format of the xaxis values in the hoverlabel
    xhoverformat="<b style='color:white'> %B %d at %H:%M</b>", 
)

#rendering the plot in html
html = fig.write_html(file="block_fees.html", auto_open=True, default_width='100%', default_height='100%', full_html=True)
#style="background-color: #11131f;"
